# import

In [2]:
# df를 위한 라이브러리
import pandas as pd 
import numpy as np

# 크롤링을 위한 라이브러리
from bs4 import BeautifulSoup as bs
import requests as req
from selenium import webdriver as wb 
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

# 시각화를 위한 라이브러리
import matplotlib.pyplot as plt
import streamlit as st 

# 한글 폰트 깨짐 방지
from matplotlib import rc
rc('font', family = 'Malgun Gothic')

# 시간 
from datetime import datetime, timedelta
import time

# 3분기 크롤링
- 7~9월 
- 검색어 : SK 하이닉스
- 네이버 뉴스

In [3]:
driver = wb.Chrome()

### 데이터 수집

In [4]:
# 뉴스 제목
titles = driver.find_elements(By.CSS_SELECTOR, 'span.sds-comps-text.sds-comps-text-ellipsis.sds-comps-text-ellipsis-1.sds-comps-text-type-headline1')
# title[0].text

for title in titles: 
    print(title.text)

#### 네이버 옵션 설정 중 날짜에 대한 url를 하나씩 설정하는 함수

In [5]:

from urllib.parse import quote

def make_news_url(query, date_obj):
    """
    query: 검색어
    date_obj: datetime.date or datetime.datetime (예: datetime(2025,7,1))
    """
    q = quote(query)

    # 날짜 두 가지 형태 만들기
    date_dot = date_obj.strftime("%Y.%m.%d")   # 2025.07.01
    date_comp = date_obj.strftime("%Y%m%d")    # 20250701

    url = (
        f"https://search.naver.com/search.naver?"
        f"ssc=tab.news.all&query={q}&sm=tab_opt&sort=0&photo=0&field=0&pd=3"
        f"&ds={date_dot}&de={date_dot}"
        f"&docid=&related=0&mynews=0&office_type=0&office_section_code=0"
        f"&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{date_comp}to{date_comp}"
        f"&is_sug_officeid=0&office_category=0&service_area=0"
    )
    return url


#### 날짜 지정

In [6]:
from datetime import datetime, timedelta

# 3분기 날짜 설정
start_date = datetime(2025, 7, 1)
end_date = datetime(2025, 9, 30)
query = "SK 하이닉스"

current = start_date

while current <= end_date:
    # URL 생성
    url = make_news_url(query, current)

    print("크롤링 날짜:", current.strftime("%Y-%m-%d"))
    print("URL:", url)

    # 페이지 이동
    driver.get(url)
    time.sleep(1)

    # 이 날의 기사 크롤링 코드 추가 가능
    # extract_articles(driver)

    # 다음 날로 이동
    current += timedelta(days=1)


크롤링 날짜: 2025-07-01
URL: https://search.naver.com/search.naver?ssc=tab.news.all&query=SK%20%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2025.07.01&de=2025.07.01&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20250701to20250701&is_sug_officeid=0&office_category=0&service_area=0
크롤링 날짜: 2025-07-02
URL: https://search.naver.com/search.naver?ssc=tab.news.all&query=SK%20%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2025.07.02&de=2025.07.02&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20250702to20250702&is_sug_officeid=0&office_category=0&service_area=0


InvalidSessionIdException: Message: invalid session id: session deleted as the browser has closed the connection
from disconnected: not connected to DevTools
  (Session info: chrome=142.0.7444.177)
Stacktrace:
Symbols not available. Dumping unresolved backtrace:
	0x7ff6fa40a235
	0x7ff6fa162630
	0x7ff6f9ef16dd
	0x7ff6f9edd205
	0x7ff6f9f02a8a
	0x7ff6f9f7a476
	0x7ff6f9f9b3d2
	0x7ff6f9f3b0ed
	0x7ff6f9f3bf63
	0x7ff6fa435d60
	0x7ff6fa42fe8a
	0x7ff6fa451005
	0x7ff6fa17d71e
	0x7ff6fa184e1f
	0x7ff6fa16b7c4
	0x7ff6fa16b97f
	0x7ff6fa1518e8
	0x7ffa01cee8d7
	0x7ffa02c6c53c


#### 스크롤을 최대로 해서 제목 수집

In [ ]:
def crawl_titles_scroll(driver, scroll_pause=1):
    """
    현재 페이지에서 스크롤 끝까지 내려가며 모든 제목을 수집하는 함수
    """
    title_selector = (
        "span.sds-comps-text.sds-comps-text-ellipsis."
        "sds-comps-text-ellipsis-1.sds-comps-text-type-headline1"
    )

    last_height = 0
    titles_collected = set()

    while True:
        # 1) 현재 제목들 추가
        titles = driver.find_elements(By.CSS_SELECTOR, title_selector)
        for t in titles:
            if t.text.strip() != "":
                titles_collected.add(t.text.strip())

        # 2) 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)

        # 3) 페이지 높이 비교하여 종료 여부 확인
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # 더 이상 로딩될 기사가 없음
            break

        last_height = new_height

    return list(titles_collected)


#### 특정 url (해당 일자) -> 제목 수집 -> df 생성 -> 저장 -> 다음 url로 이동

In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import time

start_date = datetime(2025, 8, 21)
end_date = datetime(2025, 8, 26)
query = "SK 하이닉스"

current = start_date

while current <= end_date:

    # URL 생성
    url = make_news_url(query, current)
    print("크롤링 날짜:", current.strftime("%Y-%m-%d"))
    print("URL:", url)

    # 페이지 이동
    driver.get(url)
    time.sleep(1)

    # 제목 수집 (스크롤 끝까지)
    titles = crawl_titles_scroll(driver)

    # DataFrame 생성
    df = pd.DataFrame({
        "date": [current.strftime("%Y-%m-%d")] * len(titles),
        "title": titles
    })

    # CSV 저장
    save_path = f"../mini_project1/raw_data/SK하이닉스_{current.strftime('%Y%m%d')}.csv"
    df.to_csv(save_path, encoding="utf-8-sig", index=False)

    print(f">>> 저장 완료: {save_path} | 수집 개수: {len(titles)}")
    print(df.head())

    # 다음 날짜
    current += timedelta(days=1)


#### 데이터 병합 (3분기 전체)
- `glob.glob()` : 특정 패턴과 일치하는 파일 목록을 한 번에 찾아주는 모듈

In [7]:
import glob

# 1. 합칠 csv 파일 경로 
file_list = glob.glob('./raw_data/SK하이닉스_*.csv')
print("찾은 파일:", file_list)  # 디버깅용

# 2. 각 파일을 읽어서 리스트에 담기 
df_list = []
for file in file_list :
    df = pd.read_csv(file)
    df_list.append(df)

# 3. 모두 합치기
merged_df = pd.concat(df_list, ignore_index= True)

# 4. 병합된 파일 저장
merged_df.to_csv('./preprocessed_data/SK하이닉스_3분기_merged.csv', index = False, encoding= 'utf-8-sig')
merged_df

찾은 파일: ['./raw_data\\SK하이닉스_20250701.csv', './raw_data\\SK하이닉스_20250702.csv', './raw_data\\SK하이닉스_20250703.csv', './raw_data\\SK하이닉스_20250704.csv', './raw_data\\SK하이닉스_20250705.csv', './raw_data\\SK하이닉스_20250706.csv', './raw_data\\SK하이닉스_20250707.csv', './raw_data\\SK하이닉스_20250708.csv', './raw_data\\SK하이닉스_20250709.csv', './raw_data\\SK하이닉스_20250710.csv', './raw_data\\SK하이닉스_20250711.csv', './raw_data\\SK하이닉스_20250712.csv', './raw_data\\SK하이닉스_20250713.csv', './raw_data\\SK하이닉스_20250714.csv', './raw_data\\SK하이닉스_20250715.csv', './raw_data\\SK하이닉스_20250716.csv', './raw_data\\SK하이닉스_20250717.csv', './raw_data\\SK하이닉스_20250718.csv', './raw_data\\SK하이닉스_20250719.csv', './raw_data\\SK하이닉스_20250720.csv', './raw_data\\SK하이닉스_20250721.csv', './raw_data\\SK하이닉스_20250722.csv', './raw_data\\SK하이닉스_20250723.csv', './raw_data\\SK하이닉스_20250724.csv', './raw_data\\SK하이닉스_20250725.csv', './raw_data\\SK하이닉스_20250726.csv', './raw_data\\SK하이닉스_20250727.csv', './raw_data\\SK하이닉스_20250728.csv', './raw_data\

,date,title
0,2025-07-01,[100대 기업 보고서] '한국판 ASML' 한미반도체 PBR '변곡점' [넘버스]
1,2025-07-01,"코스피, 장중 연고점 돌파 후 '주춤'…3090선 턱밑 마감"
2,2025-07-01,"[마감시황] 코스피, 장중 연고점 갱신...3089.65 마감"
3,2025-07-01,"코스피, 또 연고점 3,130선 안착 시도…삼성전자 1%대 강세 전진"
4,2025-07-01,"[마감시황] 코스피, 장중 연고점 돌파 후 후퇴...0.58% 오른 3089.65 마감"
...,...,...
21776,2025-09-30,"코스피, 소폭 '상승' 출발…네이버·카카오 '희비' 엇갈려"
21777,2025-09-30,“95%가 외국산… 車반도체 자립 생태계 만들자”
21778,2025-09-30,올 최대 순매수 연기금…슈퍼사이클 반도체에 집중 베팅
21779,2025-09-30,"'AI 열풍'에 반도체 투톱 '불기둥'···""50만닉스·11만전자 간다"""


In [ ]:
from urllib.parse import quote

def make_news_url(query, date_obj):
    """
    query: 검색어
    date_obj: datetime.date or datetime.datetime (예: datetime(2025,7,1))
    """
    q = quote(query)

    # 날짜 두 가지 형태 만들기
    date_dot = date_obj.strftime("%Y.%m.%d")   # 2025.07.01
    date_comp = date_obj.strftime("%Y%m%d")    # 20250701

    url = (
        f"https://search.naver.com/search.naver?"
        f"ssc=tab.news.all&query={q}&sm=tab_opt&sort=0&photo=0&field=0&pd=3"
        f"&ds={date_dot}&de={date_dot}"
        f"&docid=&related=0&mynews=0&office_type=0&office_section_code=0"
        f"&news_office_checked=&nso=so%3Ar%2Cp%3Afrom{date_comp}to{date_comp}"
        f"&is_sug_officeid=0&office_category=0&service_area=0"
    )
    return url


In [ ]:
from datetime import datetime, timedelta

start_date = datetime(2025, 7, 1)
end_date = datetime(2025, 9, 30)
query = "SK 하이닉스"

current = start_date

while current <= end_date:
    # URL 생성
    url = make_news_url(query, current)

    print("크롤링 날짜:", current.strftime("%Y-%m-%d"))
    print("URL:", url)

    # 페이지 이동
    driver.get(url)
    time.sleep(1)

    # 이 날의 기사 크롤링 코드 추가 가능
    # extract_articles(driver)

    # 다음 날로 이동
    current += timedelta(days=1)


크롤링 날짜: 2025-07-01
URL: https://search.naver.com/search.naver?ssc=tab.news.all&query=SK%20%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2025.07.01&de=2025.07.01&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20250701to20250701&is_sug_officeid=0&office_category=0&service_area=0
크롤링 날짜: 2025-07-02
URL: https://search.naver.com/search.naver?ssc=tab.news.all&query=SK%20%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2025.07.02&de=2025.07.02&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20250702to20250702&is_sug_officeid=0&office_category=0&service_area=0
크롤링 날짜: 2025-07-03
URL: https://search.naver.com/search.naver?ssc=tab.news.all&query=SK%20%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2025.07.03&de=2025.07.03&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_off

In [ ]:
# ★★ 테스트 날짜 설정 ★★
query = "SK 하이닉스"
date_obj = datetime(2025, 7, 2)     # ← 여기만 바꿔서 테스트하면 됨!

data = crawl_one_day(query, date_obj, driver)

df = pd.DataFrame(data, columns=["date", "title", "body"])

df.to_csv(f"../mini_project1/raw_data/SK하이닉스_{date_obj.strftime('%Y%m%d')}.csv", encoding="utf-8-sig", index=False)

print("CSV 저장 완료!")
print(df.head())


In [ ]:
def crawl_titles_scroll(driver, scroll_pause=1):
    """
    현재 페이지에서 스크롤 끝까지 내려가며 모든 제목을 수집하는 함수
    """
    title_selector = (
        "span.sds-comps-text.sds-comps-text-ellipsis."
        "sds-comps-text-ellipsis-1.sds-comps-text-type-headline1"
    )

    last_height = 0
    titles_collected = set()

    while True:
        # 1) 현재 제목들 추가
        titles = driver.find_elements(By.CSS_SELECTOR, title_selector)
        for t in titles:
            if t.text.strip() != "":
                titles_collected.add(t.text.strip())

        # 2) 스크롤 다운
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
        time.sleep(scroll_pause)

        # 3) 페이지 높이 비교하여 종료 여부 확인
        new_height = driver.execute_script("return document.body.scrollHeight")
        if new_height == last_height:
            # 더 이상 로딩될 기사가 없음
            break

        last_height = new_height

    return list(titles_collected)


In [ ]:
from datetime import datetime, timedelta
import pandas as pd
import time

start_date = datetime(2025, 8, 21)
end_date = datetime(2025, 8, 26)
query = "SK 하이닉스"

current = start_date

while current <= end_date:

    # URL 생성
    url = make_news_url(query, current)
    print("크롤링 날짜:", current.strftime("%Y-%m-%d"))
    print("URL:", url)

    # 페이지 이동
    driver.get(url)
    time.sleep(1)

    # 제목 수집 (스크롤 끝까지)
    titles = crawl_titles_scroll(driver)

    # DataFrame 생성
    df = pd.DataFrame({
        "date": [current.strftime("%Y-%m-%d")] * len(titles),
        "title": titles
    })

    # CSV 저장
    save_path = f"../mini_project1/raw_data/SK하이닉스_{current.strftime('%Y%m%d')}.csv"
    df.to_csv(save_path, encoding="utf-8-sig", index=False)

    print(f">>> 저장 완료: {save_path} | 수집 개수: {len(titles)}")
    print(df.head())

    # 다음 날짜
    current += timedelta(days=1)


크롤링 날짜: 2025-08-21
URL: https://search.naver.com/search.naver?ssc=tab.news.all&query=SK%20%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2025.08.21&de=2025.08.21&docid=&related=0&mynews=0&office_type=0&office_section_code=0&news_office_checked=&nso=so%3Ar%2Cp%3Afrom20250821to20250821&is_sug_officeid=0&office_category=0&service_area=0
>>> 저장 완료: ../mini_project1/raw_data/SK하이닉스_20250821.csv | 수집 개수: 430
         date                                         title
0  2025-08-21  제2회 대한민국 사회적 가치 페스타 ... 재난‧재해 막고, 뇌졸중 ‧백내...
1  2025-08-21     최태원 회장 "성과급 5000%, 행복 보장 안 돼…보상 집착은 근시안적"
2  2025-08-21         코스피, 기관 매수에 장중 오름폭 키워… ‘조선·방산·원전株’ 강세
3  2025-08-21     9월도 금리 내리기 힘들겠는걸…‘조정장세’ 코스피, 하방 압력 더 커...
4  2025-08-21                              8월 21일 개장 전 주요공시
크롤링 날짜: 2025-08-22
URL: https://search.naver.com/search.naver?ssc=tab.news.all&query=SK%20%ED%95%98%EC%9D%B4%EB%8B%89%EC%8A%A4&sm=tab_opt&sort=0&photo=0&field=0&pd=3&ds=2025.08.22&de=2025.08.22&docid=&r